In [1]:
import sys
sys.path
sys.path.append('/home/infres/slabbi/Master_thesis/lib/python3.8/site-packages')

In [2]:
import torch
X_train = torch.FloatTensor([0., 1., 2.]).cuda()
X_train.is_cuda

True

In [3]:
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(2)
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-16GB'

In [4]:
from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN
import torch
from copy import deepcopy
import numpy as np
from torch.nn.utils.convert_parameters import vector_to_parameters
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ExponentialLR, CosineAnnealingLR
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.utils import parameters_to_vector
from torch.distributions import Normal
import matplotlib.pyplot as plt
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as td
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange

from laplace.baselaplace import KronLaplace
from laplace.curvature import AsdlGGN
from PBB.pbb.utils import runexp
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

BATCH_SIZE = 250
TRAIN_EPOCHS = 50
DELTA = 0.025
DELTA_TEST = 0.01
PRIOR = 'learnt'

SIGMAPRIOR = 0.03

PMIN = 1e-5
KL_PENALTY = 0.1
LEARNING_RATE = 0.001
MOMENTUM = 0.95
LEARNING_RATE_PRIOR = 0.005
MOMENTUM_PRIOR = 0.99

# note the number of MC samples used in the paper is 150.000, which usually takes a several hours to compute
MC_SAMPLES = 1000

True


In [5]:
runexp('mnist', 'practical_bayes_deep', PRIOR, 'cnn', SIGMAPRIOR, PMIN, LEARNING_RATE, MOMENTUM, LEARNING_RATE_PRIOR, MOMENTUM_PRIOR, delta=DELTA, delta_test=DELTA_TEST, mc_samples=MC_SAMPLES, train_epochs=TRAIN_EPOCHS, device=DEVICE, prior_epochs=20, perc_train=1.0, perc_prior=0.5, verbose=True, dropout_prob=0.2)

  5%|██████                                                                                                                  | 1/20 [00:05<01:37,  5.15s/it]

-Epoch 0.00000, Train loss: 0.94177, Train err:  0.23747



 10%|████████████                                                                                                            | 2/20 [00:09<01:26,  4.78s/it]

-Epoch 1.00000, Train loss: 0.17676, Train err:  0.05087



 15%|██████████████████                                                                                                      | 3/20 [00:14<01:19,  4.70s/it]

...

-Epoch 18.00000, Train loss: 0.00006, Train err:  0.00000



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:31<00:00,  4.56s/it]

-Epoch 19.00000, Train loss: 0.00005, Train err:  0.00000


-Prior: Test loss: 2.68785, Test err:  0.01140
Parameter containing:
tensor([[ 0.0008,  0.0077, -0.0092,  ..., -0.0095,  0.0090, -0.0098],
        [-0.0065, -0.0062, -0.0115,  ..., -0.0070,  0.0020,  0.0060],
        [ 0.0025, -0.0012,  0.0037,  ...,  0.0094, -0.0037,  0.0109],
        ...,
        [-0.0103, -0.0070, -0.0097,  ...,  0.0007,  0.0034,  0.0051],
        [-0.0038,  0.0060,  0.0061,  ..., -0.0065,  0.0074, -0.0093],
        [ 0.0033, -0.0083,  0.0107,  ..., -0.0006,  0.0042, -0.0075]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[-0.0340, -0.6054,  0.1580,  ..., -0.0537,  0.3178, -0.0463],
        [-0.0490,  0.1710,  0.4102,  ..., -0.2281, -0.2717, -0.0492],
        [-0.1322, -0.0387,  0.4054,  ..., -0.3379,  0.1791,  0.0715],
        ...,
        [-0.0098, -0.0936, -0.1207,  ..., -0.0463, -0.1073, -0.0630],
        [-0.1503,  0.3119, -0.0979,  ...,  0.1922,  0.3070,  0.0097],
        [-0.3341,  0.6416, -0.2963,  ...,  0.2909,  0.0229, -0.1013]

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.